<a href="https://colab.research.google.com/github/kykim0/tinkering/blob/main/PowerOfMCv0_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Power of Monte Carlo Dropout - LearnML.co.uk

First Created: 21.06.20



In this workbook we introduce an incredibly useful, yet largely overlooked, modification of neural networks that use dropout for regularisation purposes. The motivation of this tutorial is to make you aware of Monte Carlo Dropout, and to inspire you to try it out for yourself!

## 1. Imports and Setup

### 1.1. Imports

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.losses import mean_absolute_error, mean_squared_error
from tensorflow.keras.optimizers import Nadam, Adam
from tensorflow.keras.initializers import lecun_normal

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### 1.2. Data Setup

In [ ]:
df_train = pd.read_csv("/content/sample_data/california_housing_train.csv")
df_test = pd.read_csv("/content/sample_data/california_housing_test.csv")

In [ ]:
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [ ]:
df_train = df_train.drop(["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households"], axis=1)
df_test = df_test.drop(["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households"], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train = df_train.drop(["median_house_value"], axis=1).values
y_train = df_train.median_house_value.values
X_test = df_test.drop(["median_house_value"], axis=1).values
y_test = df_test.median_house_value.values

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

## 2. No Monte Carlo Dropout

### 2.1. Neural Network

In [ ]:
model_NN = Sequential([InputLayer(input_shape=(1,)),
             Dropout(0.4),          
			 Dense(1000, activation="selu", kernel_initializer="lecun_normal"),
             Dropout(0.4),
             Dense(2000, activation="selu", kernel_initializer="lecun_normal"),
             Dropout(0.4),
             Dense(2000, activation="selu", kernel_initializer="lecun_normal"),
			 Dropout(0.4),
             Dense(1000, activation="selu", kernel_initializer="lecun_normal"),
			 Dropout(0.4),
             Dense(1, activation=None)
			 ])
print(model_NN)

In [ ]:
model_NN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 1000)              2000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              2002000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 2000)              0

In [ ]:
model_NN.compile(loss="mean_squared_error",
              optimizer="Nadam",
              metrics=["mean_squared_error"])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

### 2.2. Training

In [ ]:
history_NN = model_NN.fit(X_train_norm, y_train, epochs=201, validation_split=0.2, callbacks=[callback])

Epoch 1/200
425/425 [==============================] - 2s 6ms/step - loss: 12860594176.0000 - mean_squared_error: 12860594176.0000 - val_loss: 10660553728.0000 - val_mean_squared_error: 10660553728.0000
Epoch 2/200
425/425 [==============================] - 2s 6ms/step - loss: 9110819840.0000 - mean_squared_error: 9110819840.0000 - val_loss: 9721359360.0000 - val_mean_squared_error: 9721359360.0000
Epoch 3/200
425/425 [==============================] - 2s 5ms/step - loss: 9060517888.0000 - mean_squared_error: 9060517888.0000 - val_loss: 11310814208.0000 - val_mean_squared_error: 11310814208.0000
Epoch 4/200
425/425 [==============================] - 2s 5ms/step - loss: 9119346688.0000 - mean_squared_error: 9119346688.0000 - val_loss: 10608936960.0000 - val_mean_squared_error: 10608936960.0000


### 2.3. Plots

In [ ]:
# weight adjustment due to dropout
WEIGHTS = model_NN.get_weights()
WEIGHTS[0] *= 1/0.4
model_NN.set_weights(WEIGHTS)

In [ ]:
df_train.head(2)

,median_income,median_house_value
0,1.4936,66900.0
1,1.8200,80100.0


In [ ]:
predictions = model_NN.predict(X_test_norm)

In [ ]:
df_test["predictions"] = predictions

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_test.median_income, y=df_test.median_house_value, mode="markers"))
fig.add_trace(go.Scatter(x=df_test.median_income, y=df_test.predictions, mode="markers"))

fig.show()

## 3. Monte Carlo Dropout

### 3.1. Neural Network

In [ ]:
class MCDropout(Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [ ]:
model_MC = Sequential([InputLayer(input_shape=(1,)),
             MCDropout(0.4),          
			 Dense(1000, activation="selu", kernel_initializer="lecun_normal"),
             MCDropout(0.4),
             Dense(2000, activation="selu", kernel_initializer="lecun_normal"),
             MCDropout(0.4),
             Dense(2000, activation="selu", kernel_initializer="lecun_normal"),
			 MCDropout(0.4),
             Dense(1000, activation="selu", kernel_initializer="lecun_normal"),
			 MCDropout(0.4),
             Dense(1, activation=None)
			 ])

In [ ]:
model_MC.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mc_dropout (MCDropout)       (None, 1)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              2000      
_________________________________________________________________
mc_dropout_1 (MCDropout)     (None, 1000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2000)              2002000   
_________________________________________________________________
mc_dropout_2 (MCDropout)     (None, 2000)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
mc_dropout_3 (MCDropout)     (None, 2000)             

In [ ]:
model_MC.compile(loss="mean_squared_error",
              optimizer="Nadam",
              metrics=["mean_squared_error"])

### 3.2. Training

In [ ]:
history_MC = model_MC.fit(X_train_norm, y_train, epochs=200, validation_split=0.2, callbacks=[callback])

Epoch 1/200
425/425 [==============================] - 2s 6ms/step - loss: 12894774272.0000 - mean_squared_error: 12894774272.0000 - val_loss: 13365480448.0000 - val_mean_squared_error: 13365480448.0000
Epoch 2/200
425/425 [==============================] - 2s 6ms/step - loss: 9197340672.0000 - mean_squared_error: 9197340672.0000 - val_loss: 13993886720.0000 - val_mean_squared_error: 13993886720.0000
Epoch 3/200
425/425 [==============================] - 2s 6ms/step - loss: 9143620608.0000 - mean_squared_error: 9143620608.0000 - val_loss: 12880267264.0000 - val_mean_squared_error: 12880267264.0000
Epoch 4/200
425/425 [==============================] - 2s 6ms/step - loss: 9191550976.0000 - mean_squared_error: 9191550976.0000 - val_loss: 11997221888.0000 - val_mean_squared_error: 11997221888.0000
Epoch 5/200
425/425 [==============================] - 2s 6ms/step - loss: 9207301120.0000 - mean_squared_error: 9207301120.0000 - val_loss: 15540974592.0000 - val_mean_squared_error: 1554097459

### 3.3. Training

In [ ]:
# weight adjustment due to dropout
WEIGHTS = model_MC.get_weights()
WEIGHTS[0] *= 1/0.4
model_MC.set_weights(WEIGHTS)

In [ ]:
y_pred_stack = np.stack([model_MC.predict(X_test_norm) for sample in range(10)])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_test.median_income, y=df_test.median_house_value, mode="markers"))
for i in range(len(y_pred_stack)):

    fig.add_trace(go.Scatter(x=df_test.median_income, y=y_pred_stack[i].reshape(-1,), mode="markers", opacity=0.1,  marker=dict(
            color='Red')))

fig.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Histogram(x=df_train["median_income"].values, opacity=0.3, marker=dict(color="Gold")), secondary_y=True)

fig.add_trace(go.Scatter(x=df_test.median_income, y=df_test.median_house_value, mode="markers", marker=dict(color="Orange")))
for i in range(len(y_pred_stack)):

    fig.add_trace(go.Scatter(x=df_test.median_income, y=y_pred_stack[i].reshape(-1,), mode="markers", opacity=0.1,  marker=dict(
            color='Red')))

fig.add_trace(go.Scatter(x=df_test.median_income, y=y_pred_stack.mean(axis=0).reshape(-1,), mode="markers", opacity=.7,  marker=dict(
            color='Green')))

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> House Price", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Number of datapoints", secondary_y=True)
fig.update_xaxes(title_text="Household Income")
fig.show()